참고 사이트 : https://yunwoong.tistory.com/153

In [17]:
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
from matplotlib import pyplot as plt

import uuid
import json
import time
import cv2
import requests

In [18]:
def plt_imshow(title='image', img=None, figsize=(8, 5)):
    plt.figure(figsize=figsize)

    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)

            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])

        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()


def put_text(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)

    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
    elif platform.system() == "Linux":  # 원본에서 이 부분만 수정함
        # font = 'Ubuntu-R.ttf'
        font = 'd2coding.ttf'

    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)

    draw.text((x, y), text, font=image_font, fill=color)

    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

    return opencv_image

In [19]:
path = "./images/my_profile.png"
files = [
    ("file", open(path, "rb")),
]

In [20]:
api_url = "https://9upvb67beq.apigw.ntruss.com/custom/v1/28024/7cb9fc59c2d43289019b8324b342505401dcaebbc780f57c7030a8fa4a1afa5d/general"
secret_key = "T0FsWlRPdGZ4TUxHd2phUVd2cHduQklWU2t5cVRzUnU="

In [21]:
request_json = {
    'images': [
        {
            'format': path.split(".")[-1],
            'name': path.split(".")[0]
        }
    ],
    'requestId': str(uuid.uuid4()),
    'version': "V2",
    'timestamp': int(round(time.time() * 1000)),
}

In [22]:
message = json.dumps(request_json).encode("utf-8")
message

b'{"images": [{"format": "png", "name": ""}], "requestId": "38e84c4b-fe6b-4301-b90f-a3fee7f8df8d", "version": "V2", "timestamp": 1706672583281}'

In [23]:
payload = {
    "message": message
}

In [24]:
headers = {
    'X-OCR-SECRET': secret_key,
}

In [25]:
response = requests.request(method="POST",
                            url=api_url,
                            headers=headers,
                            data=payload,
                            files=files)

ValueError: read of closed file

In [ ]:
result = response.json()

In [ ]:
result

In [ ]:
field_list = result["images"][0]["fields"]

for i in range(len(field_list)):
    print(field_list[i]["inferText"])

In [ ]:
img = cv2.imread(path)
roi_img = img.copy()

for field in result['images'][0]['fields']:
    text = field['inferText']
    vertices_list = field['boundingPoly']['vertices']
    pts = [tuple(vertice.values()) for vertice in vertices_list]
    topLeft = [int(_) for _ in pts[0]]
    topRight = [int(_) for _ in pts[1]]
    bottomRight = [int(_) for _ in pts[2]]
    bottomLeft = [int(_) for _ in pts[3]]

    cv2.line(roi_img, topLeft, topRight, (0, 255, 0), 2)
    cv2.line(roi_img, topRight, bottomRight, (0, 255, 0), 2)
    cv2.line(roi_img, bottomRight, bottomLeft, (0, 255, 0), 2)
    cv2.line(roi_img, bottomLeft, topLeft, (0, 255, 0), 2)
    roi_img = put_text(roi_img, text, topLeft[0], topLeft[1] - 10, font_size=30)

    print(text)

plt_imshow(["Original", "ROI"], [img, roi_img], figsize=(16, 10))